# Quick test of different ways of estimating Kappas across subejcts and parcels



In [1]:
# dependencies
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('..'))

import numpy as np
import torch as pt
import matplotlib.pyplot as plt
from copy import copy,deepcopy

import HierarchBayesParcel.emissions as em
import HierarchBayesParcel.arrangements as ar


## make some artifical data

In [2]:

n_subj = 4
P = 1000
N = 10
K = 5
U = pt.randint(low=0, high=K, size=(n_subj, P))
U_hat = ar.expand_mn(U,K)
emission_model = em.MixVMF(K, N, P, n_subj)
emission_model.kappa=pt.tensor(20)
Y = emission_model.sample(U)

In [3]:
m = deepcopy(emission_model)
m.random_params()

## Do an M-step by hand (original)

In [8]:
JU = pt.sum(U_hat,dim=2)   # (num_sub, K)
 # Calculate YU = \sum_i\sum_k<u_i^k>y_i # (num_sub, N, K)
YU = pt.matmul(Y, pt.transpose(U_hat, 1, 2))

# If the subjects are weighted differently
r_norm2 = pt.sum(YU ** 2, dim=1, keepdim=True) # (num_sub, 1, K)


tensor([[[20997.5862, 26533.2157, 26438.9563, 26213.7248, 26459.2808]],

        [[20087.1616, 21316.7961, 30317.8154, 27559.5107, 27791.7369]],

        [[28180.9209, 19115.2830, 28567.6862, 28631.3266, 23381.9699]],

        [[18605.6972, 29121.6877, 27025.2887, 27579.1671, 24978.5842]]])

In [9]:
pt.sqrt(r_norm2.squeeze())/JU

tensor([[0.7962, 0.7985, 0.7893, 0.7937, 0.7974],
        [0.7830, 0.7978, 0.8024, 0.7905, 0.7976],
        [0.8032, 0.7992, 0.7861, 0.8019, 0.7964],
        [0.8024, 0.7864, 0.8019, 0.7984, 0.7902]])

In [6]:
# Overall V estimation
V = pt.sum(YU,dim=0) / JU.sum(dim=0, keepdim=True)
print(pt.sqrt(pt.sum(V**2, dim=0, keepdim=True)))

tensor([[0.7940, 0.7970, 0.7973, 0.7970, 0.7973]])


In [7]:
# Subject-equal weighting V estimation
V=pt.nanmean(YU / JU.unsqueeze(1),dim=0)
print(pt.sqrt(pt.sum(V**2, dim=0, keepdim=True)))

tensor([[0.7938, 0.7970, 0.7974, 0.7971, 0.7974]])


In [8]:
# Overall kappa
yu = pt.sum(YU,dim=0)
r=pt.sum(pt.sqrt(pt.sum(yu**2, dim=0)))/ pt.sum(JU)
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor(0.7965) tensor(20.4098)


In [9]:
# Subject-specific kappa
r = pt.sum(pt.sqrt(pt.sum(YU**2, dim=1)),dim=1)/pt.sum(JU,dim=1)
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor([0.7955, 0.7984, 0.7941, 0.8011]) tensor([20.2941, 20.6166, 20.1457, 20.9284])


In [10]:
# Region-specfic kappa
yu = pt.sum(YU,dim=0)
r=pt.sqrt(pt.sum(yu**2, dim=0))/ pt.sum(JU,dim=0)
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor([0.7940, 0.7970, 0.7973, 0.7970, 0.7973]) tensor([20.1350, 20.4602, 20.4958, 20.4650, 20.4911])


In [11]:
# Region and subject-specific kappa
r=pt.sqrt(pt.sum(YU**2, dim=1))/ JU
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor([[0.7905, 0.7955, 0.7899, 0.7989, 0.8020],
        [0.7998, 0.7952, 0.8021, 0.7979, 0.7967],
        [0.7877, 0.7896, 0.8044, 0.7955, 0.7940],
        [0.8002, 0.8098, 0.7962, 0.7996, 0.7997]]) tensor([[19.7541, 20.2950, 19.6932, 20.6769, 21.0319],
        [20.7729, 20.2561, 21.0469, 20.5564, 20.4271],
        [19.4635, 19.6669, 21.3143, 20.2994, 20.1252],
        [20.8219, 21.9854, 20.3734, 20.7525, 20.7627]])


## Do the M-step by hand with using the actual V-vectors

In [40]:
yu = pt.sum(YU,dim=0)
v = pt.sum(YU, dim=0) / pt.sum(JU,dim=0)
V = v/pt.sqrt(pt.sum(v**2, dim=0, keepdim=True))
r1 = pt.sum(pt.sqrt(pt.sum(yu**2, dim=0)))/ pt.sum(JU)
r2 = pt.mean(pt.sqrt(pt.sum(v**2, dim=0)))
r3 = pt.mean(pt.sum(v*V,dim=0))
print(r1,r2,r3)

tensor(0.7947) tensor(0.7947) tensor(0.7947)


tensor([[-0.0287,  0.1604, -0.2923, -0.0198, -0.4117],
        [-0.0873, -0.5630, -0.1184,  0.4544,  0.2736],
        [ 0.0101, -0.3681,  0.4004, -0.3341,  0.1215],
        [-0.2620,  0.2173, -0.2429, -0.0793,  0.0644],
        [-0.2914,  0.0079, -0.1076,  0.1028, -0.1801],
        [-0.2599,  0.1698, -0.0827,  0.1002,  0.1537],
        [-0.1356,  0.1620, -0.3526, -0.3068, -0.0989],
        [ 0.5308,  0.0057,  0.0278, -0.1587,  0.1548],
        [ 0.2566,  0.0550, -0.3629, -0.0148, -0.3688],
        [-0.1923, -0.2175, -0.1913,  0.4098,  0.3767]])